First we need to read:
1. the coordinates of the atoms belonging in the primitive cell
2. the primitive cell vectors
The easiest way to do so is to use Mercury: open the cif file of the system under study, AutoEdit the structure (optional but advisable), enable Packing, and save as mol2 file. The generated mol2 file contains all necessary information.

In [ ]:
import math
def read_mol2(file):
    fp=open(file,'r')
    mol2=fp.readlines()
    fp.close()
    species=[]
    x=[]
    y=[]
    z=[]
    for line_counter,line in enumerate(mol2):
        if line.find('@<TRIPOS>MOLECULE\n')==0:
            name=mol2[line_counter+1]
            atoms=int(mol2[line_counter+2].split()[0])
        if line.find('@<TRIPOS>ATOM\n')==0:
            for i in range(atoms):
                x.append(float(mol2[line_counter+i+1].split()[2]))
                y.append(float(mol2[line_counter+i+1].split()[3]))
                z.append(float(mol2[line_counter+i+1].split()[4]))
                species.append(mol2[line_counter+i+1].split()[5])
        if line.find('@<TRIPOS>CRYSIN\n')==0:
            a=float(mol2[line_counter+1].split()[0])
            b=float(mol2[line_counter+1].split()[1])
            c=float(mol2[line_counter+1].split()[2])
            alpha=float(mol2[line_counter+1].split()[3])
            beta=float(mol2[line_counter+1].split()[4])
            gamma=float(mol2[line_counter+1].split()[5])
    ax=a
    ay=0.0
    az=0.0
    bx=b*math.cos(gamma*math.pi/180.0)
    by=b*math.sin(gamma*math.pi/180.0)
    bz=0.0
    cx=c*math.cos(beta*math.pi/180.0)
    cy=(b*c*math.cos(alpha*math.pi/180.0)-bx*cx)/by
    cz=math.sqrt(c*c-cx*cx-cy*cy)
    a_vector=[ax,ay,az]
    b_vector=[bx,by,bz]
    c_vector=[cx,cy,cz]
    a_vector=[round(i,10) for i in a_vector]
    b_vector=[round(i,10) for i in b_vector]
    c_vector=[round(i,10) for i in c_vector]
    species=[i.split('.')[0] for i in species]
    return [atoms,species,x,y,z,a_vector,b_vector,c_vector]

In [ ]:
# use dictionary to define system parameters
cp2k_init={
    'filename': 'RESORA09.mol2',
    'BASIS_SET': 'DZVP-GTH',
    'POTENTIAL': 'GTH-PBE'
}
# get coords and cell from mol2 file
atoms,species,x,y,z,a,b,c=read_mol2(cp2k_init['filename'])
# write subsys file
repeat=[1,1,1]
unique_species=[i for i in set(species)]
with open('subsys.include',mode='w') as fp:
    print(f' &SUBSYS', file=fp)
    print(f'  &CELL', file=fp)
    print(f'   A {" ".join(list(map(str,a)))}', file=fp)
    print(f'   B {" ".join(list(map(str,b)))}', file=fp)
    print(f'   C {" ".join(list(map(str,c)))}', file=fp)
    print(f'   PERIODIC XYZ', file=fp)
    print(f'   MULTIPLE_UNIT_CELL {" ".join(list(map(str,repeat)))}', file=fp)
    print(f'  &END CELL', file=fp)
    print(f'  &TOPOLOGY', file=fp)
    print(f'   MULTIPLE_UNIT_CELL {" ".join(list(map(str,repeat)))}', file=fp)
    print(f'  &END TOPOLOGY', file=fp)
    print(f'  &COORD', file=fp)
    for i in range(atoms):
        print(f'   {species[i]} {x[i]} {y[i]} {z[i]}', file=fp)
    print(f'  &END COORD', file=fp)
    for element in unique_species:
        print(f'  &KIND {element}', file=fp)
        print(f'   ELEMENT {element}', file=fp)
        print(f'   BASIS_SET {cp2k_init["BASIS_SET"]}', file=fp)
        print(f'   POTENTIAL {cp2k_init["POTENTIAL"]}', file=fp)
        print(f'  &END KIND', file=fp)
    print(f' &END SUBSYS', file=fp)